# Examples of scheduled circuit  (circuit with duration)

In [1]:
from qiskit import QuantumCircuit
from qiskit.test.mock.backends import FakeParis
backend = FakeParis()

## Manual construction of T2 experiment

In [2]:
qc = QuantumCircuit(1, 1, name="t2_experiment")
qc.h(0)
qc.delay(1000, 0, unit='ns')
qc.h(0)
qc.measure(0, 0)
qc.draw()

┌───┐┌───────┐┌───┐┌─┐
q_0: ┤ H ├┤ DELAY ├┤ H ├┤M├
     └───┘└───────┘└───┘└╥┘
c_0: ════════════════════╩═

In [3]:
print(qc.schedule(backend))

     |--------------------------------------------------------------|
q_0: | U2(0,pi)[160] | DELAY[4500] | U2(0,pi)[160] | MEASURE[19200] |
     |--------------------------------------------------------------|


## Schedule Bell experiment with scheduling passes

In [4]:
from qiskit import transpile
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit.transpiler.passes.scheduling.asap import ASAPSchedule
from qiskit.transpiler.passes.scheduling.alap import ALAPSchedule

In [5]:
qc = QuantumCircuit(2, name="bell")
qc.h(0)
qc.delay(1000, 1, unit='ns')
qc.cx(0,1)
qc.draw()

┌───┐       
q_0: ──┤ H ├────■──
     ┌─┴───┴─┐┌─┴─┐
q_1: ┤ DELAY ├┤ X ├
     └───────┘└───┘

In [6]:
print(qc.schedule(backend))

     |----------------------------------------|
q_0: | DELAY[4340] | U2(0,pi)[160] | CX[1408] |
     |----------------------------------------|
q_1: | DELAY[4500]                 | CX[1408] |
     |----------------------------------------|


`qc.schedule(backend)` is defined as follows.

In [7]:
transpiled = transpile(qc, backend=backend, optimization_level=0, basis_gates=['u1', 'u2', 'u3', 'cx', 'delay'])
dag = circuit_to_dag(transpiled)
dag_with_delays = ALAPSchedule(backend).run(dag)
scheduled = dag_to_circuit(dag_with_delays)
print(scheduled)

     |----------------------------------------|
q_0: | DELAY[4340] | U2(0,pi)[160] | CX[1408] |
     |----------------------------------------|
q_1: | DELAY[4500]                 | CX[1408] |
     |----------------------------------------|


###  ASAP or ALAP Schedule

In [8]:
qc = QuantumCircuit(2, name="h2")
qc.h(0)
qc.x(1)
dag = circuit_to_dag(transpile(qc, backend=backend, optimization_level=0))
qc.draw()

┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ X ├
     └───┘

In [9]:
#ASAP
dag_with_delays = ASAPSchedule(backend).run(dag)
scheduled = dag_to_circuit(dag_with_delays)
print(scheduled)

     |----------------------------|
q_0: | U2(0,pi)[160] | DELAY[160] |
     |----------------------------|
q_1: | U3(pi,0,pi)[320]           |
     |----------------------------|


In [10]:
#ALAP
dag_with_delays = ALAPSchedule(backend).run(dag)
scheduled = dag_to_circuit(dag_with_delays)
print(scheduled)

     |----------------------------|
q_0: | DELAY[160] | U2(0,pi)[160] |
     |----------------------------|
q_1: | U3(pi,0,pi)[320]           |
     |----------------------------|


##  Issues identified

1. We cannot get length of measure from backend.properties (I struggled [here](https://github.com/itoko/qiskit-sdk-py/blob/1c7bc7f7c8299dab7abc9f0ebb3ef70e496267b3/qiskit/transpiler/passes/scheduling/asap.py#L38))
2. As Lauren suggested, we need more useful helper function (or class) for getting duration (gate length) of each instruction.
3. Should we output delay when qasm()? Will delay be included in QASM3? (related to https://github.com/Qiskit/qiskit-terra/issues/4312)
4. How to distinguish regular circuit and scheduled circuit in the code level: `if self.duration` or `if isinstance(qc, ScheduleCircuit)`

##  Questions

1. Can you give me any use case where operations of scheduled circuits are required? (When do we really need operations more than `append()`?)

## T2 experiment with dynamical decoupling?

In [11]:
N = 2

In [12]:
dd = QuantumCircuit(1, name="XYXY")
dd.x(0)
dd.y(0)
dd.x(0)
dd.y(0)
dd.draw()

┌───┐┌───┐┌───┐┌───┐
q_0: ┤ X ├┤ Y ├┤ X ├┤ Y ├
     └───┘└───┘└───┘└───┘

In [13]:
t2dd = QuantumCircuit(1, 1, name="t2_with_dynamical_decoupling")
t2dd.h(0)
t2dd.append(dd.to_instruction().repeat(N), qargs=[0])
t2dd.h(0)
t2dd.measure(0, 0)
t2dd.draw()

┌───┐┌────────┐┌───┐┌─┐
q_0: ┤ H ├┤ XYXY*2 ├┤ H ├┤M├
     └───┘└────────┘└───┘└╥┘
c_0: ═════════════════════╩═

In [14]:
t2 = QuantumCircuit(1, 1, name="t2_with_delay")
t2.h(0)
t2.delay(N * dd.schedule(backend).duration)
t2.h(0)
t2.measure(0, 0)
t2.draw()

┌───┐┌───────┐┌───┐┌─┐
q_0: ┤ H ├┤ DELAY ├┤ H ├┤M├
     └───┘└───────┘└───┘└╥┘
c_0: ════════════════════╩═

In [15]:
print(t2dd.schedule(backend))

     |--------------------------------------------------------------------------
q_0: | U2(0,pi)[160] | U3(pi,0,pi)[320] | U3(pi,pi/2,pi/2)[320] | U3(pi,0,pi)[32
     |--------------------------------------------------------------------------
--------------------------------------------------------------------------------
0] | U3(pi,pi/2,pi/2)[320] | U3(pi,0,pi)[320] | U3(pi,pi/2,pi/2)[320] | U3(pi,0,
--------------------------------------------------------------------------------
------------------------------------------------------------------|
pi)[320] | U3(pi,pi/2,pi/2)[320] | U2(0,pi)[160] | MEASURE[19200] |
------------------------------------------------------------------|


In [16]:
print(t2.schedule(backend))

     |--------------------------------------------------------------|
q_0: | U2(0,pi)[160] | DELAY[2560] | U2(0,pi)[160] | MEASURE[19200] |
     |--------------------------------------------------------------|


###  Schedule passes convert the duration units into dt (values are rounded)

In [17]:
bell = QuantumCircuit(2, name="bell_with_manual_delay_ns")
bell.h(0)
bell.delay(999, 1, unit='ns')
bell.cx(0,1)
print(bell.schedule(backend))

     |----------------------------------------|
q_0: | DELAY[4336] | U2(0,pi)[160] | CX[1408] |
     |----------------------------------------|
q_1: | DELAY[4496]                 | CX[1408] |
     |----------------------------------------|


/Users/itoko/swd/qiskit-dev/qiskit-sdk-py/qiskit/transpiler/passes/scheduling/asap.py:58: UserWarning: Duration of delay is rounded to 4496 dt = 9.991111e-07 s from 9.990000e-07
  UserWarning)


In [18]:
bell = QuantumCircuit(2, name="bell_with_manual_delay_unitless")
bell.h(0)
bell.delay(999, 1)
bell.cx(0,1)
print(bell.schedule(backend))

     |---------------------------------------|
q_0: | DELAY[839] | U2(0,pi)[160] | CX[1408] |
     |---------------------------------------|
q_1: | DELAY[999]                 | CX[1408] |
     |---------------------------------------|
